In [11]:
import pandas as pd
import glob

In [12]:
import seaborn as sns
import matplotlib.pyplot as plt

In [13]:
%pip install --quiet optuna

Note: you may need to restart the kernel to use updated packages.


In [14]:
import optuna
import os

/home/duarbrun/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
optuna.__version__

'4.2.1'

In [ ]:
os.chdir("drive/MyDrive/Mestrado/GraphColor/results_abc_v1")

In [ ]:
!ls

## ABC V1

In [ ]:
# Definir o caminho da pasta onde estão os arquivos TXT
folder_path = "results_abc/random/"  # Substitua pelo caminho correto

# Lista todos os arquivos TXT na pasta
txt_files = glob.glob(folder_path + "*.txt")

# Definir os nomes das colunas (ajuste conforme os dados)
columns = ["fitness", "execution_time", "num_bees", "limit", "max_iter",
           "filename", "num_vertices", "num_edges"]

txt_files

In [ ]:
# Lista para armazenar os DataFrames individuais
dfs = []

# Ler cada arquivo TXT e adicionar ao DataFrame
for file in txt_files:
    df = pd.read_csv(file, sep=";", names=columns, header=None)
    df['graph'] = file.split('/')[2].replace('.txt', '')
    dfs.append(df)

# Concatenar todos os DataFrames em um só
df_final = pd.concat(dfs, ignore_index=True)

# Exibir as primeiras linhas do DataFrame consolidado
df_final.head()

In [ ]:
df_final[df_final['graph']==None]

In [ ]:
# Normalizar a fitness dentro de cada grafo (opcional, mas pode ajudar)
df_final["fitness_norm"] = df_final.groupby("graph")["fitness"].transform(lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else 0)

df_final

In [ ]:
def objective(trial):
    """Função objetivo para o Optuna, buscando a melhor configuração de parâmetros já testada."""

    # Optuna irá escolher um conjunto de parâmetros dentro dos valores existentes
    num_bees = trial.suggest_categorical("num_bees", df_final["num_bees"].unique().tolist())
    limit = trial.suggest_categorical("limit", df_final["limit"].unique().tolist())
    max_iter = trial.suggest_categorical("max_iter", df_final["max_iter"].unique().tolist())

    # Filtrar apenas as execuções com esses parâmetros
    subset = df_final[(df_final["num_bees"] == num_bees) &
                (df_final["limit"] == limit) &
                (df_final["max_iter"] == max_iter)]

    # Média das fitness normalizadas como critério de avaliação
    fitness_score = subset["fitness_norm"].mean()

    return fitness_score  # O Optuna busca minimizar esse valor

In [ ]:
# Criar e rodar a otimização
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)  # Ajuste o número de trials conforme necessário

# Exibir os melhores parâmetros encontrados
print("Melhores parâmetros encontrados:")
print(study.best_params)

## ABC V2

### Calibrar parâmetros

In [16]:
!ls

 Optuna.ipynb			       main_test_greedy
 Optuna.ipynb:Zone.Identifier	       main_test_greedy.cpp
 README.md			       main_test_methods.cpp
 documentacao.md		       resultado_calibrar_parametro
 include			       results_abc
 instance_calibrar.csv		       results_abc_2
 instances			       results_grasp
 instances_info.csv		       results_greedy
'instances_with_k copy.csv'	       run_abc
 instances_with_k.csv		       run_abc_calib
 instances_with_k2.csv		       run_grasp
 instances_with_k_abc_2.csv	       run_greedy
 instances_with_k_no_greedy_zero.csv   run_main_abc.cpp
 lib				       run_main_abc_calibrar_parametros.cpp
 main				       run_main_abc_until_zero.cpp
 main.cpp			       run_main_grasp.cpp
 main_test_grasp		       run_main_greedy.cpp
 main_test_grasp.cpp		       shell_scripts


In [17]:
columns = ['conflitos', 'tempo_us', 'iter_found_best', 'num_bees', 
           'limit', 'max_iter', 'filename', 'rcl_size', 'random_search',
           'num_vertices', 'num_edges' ]
df = pd.read_csv('resultado_calibrar_parametro/DSJC500.5.txt', sep=';', names=columns, header=None)
df.head()

,conflitos,tempo_us,iter_found_best,num_bees,limit,max_iter,filename,rcl_size,random_search,num_vertices,num_edges
0,165,899234,0,50,5,20,instances/DSJC500.5.col,1,0,500,62624
1,165,875770,0,50,5,20,instances/DSJC500.5.col,1,0,500,62624
2,169,890426,0,50,5,20,instances/DSJC500.5.col,1,0,500,62624
3,158,878147,0,50,5,20,instances/DSJC500.5.col,1,0,500,62624
4,167,883116,0,50,5,20,instances/DSJC500.5.col,1,0,500,62624


In [18]:
# df['fitness_norm'] = df["fitness"].transform(lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else 0)

In [21]:
def objective_v2(trial):
    """Função objetivo para o Optuna, buscando a melhor configuração de parâmetros já testada."""

    # Optuna irá escolher um conjunto de parâmetros dentro dos valores existentes
    num_bees = trial.suggest_categorical("num_bees", df["num_bees"].unique().tolist())
    limit = trial.suggest_categorical("limit", df["limit"].unique().tolist())
    max_iter = trial.suggest_categorical("max_iter", df["max_iter"].unique().tolist())

    # Filtrar apenas as execuções com esses parâmetros
    subset = df[(df["num_bees"] == num_bees) &
                (df["limit"] == limit) &
                (df["max_iter"] == max_iter)]

    # Média das fitness normalizadas como critério de avaliação
    fitness_score = subset["conflitos"].mean()

    return fitness_score  # O Optuna busca minimizar esse valor

In [22]:
# Criar e rodar a otimização
study = optuna.create_study(direction="minimize")
study.optimize(objective_v2, n_trials=100)  # Ajuste o número de trials conforme necessário

# Exibir os melhores parâmetros encontrados
print("Melhores parâmetros encontrados:")
print(study.best_params)

[I 2025-03-03 23:25:02,187] A new study created in memory with name: no-name-35c0fa86-c275-4a17-ae5a-d1300c8a65fa
[I 2025-03-03 23:25:02,197] Trial 0 finished with value: 162.2 and parameters: {'num_bees': 300, 'limit': 10, 'max_iter': 80}. Best is trial 0 with value: 162.2.
[I 2025-03-03 23:25:02,203] Trial 1 finished with value: 160.65 and parameters: {'num_bees': 300, 'limit': 10, 'max_iter': 70}. Best is trial 1 with value: 160.65.
[I 2025-03-03 23:25:02,206] Trial 2 finished with value: 163.95 and parameters: {'num_bees': 100, 'limit': 50, 'max_iter': 20}. Best is trial 1 with value: 160.65.
[I 2025-03-03 23:25:02,209] Trial 3 finished with value: 159.3 and parameters: {'num_bees': 300, 'limit': 25, 'max_iter': 50}. Best is trial 3 with value: 159.3.
[I 2025-03-03 23:25:02,212] Trial 4 finished with value: 163.1 and parameters: {'num_bees': 100, 'limit': 25, 'max_iter': 50}. Best is trial 3 with value: 159.3.
[I 2025-03-03 23:25:02,216] Trial 5 finished with value: 161.0 and param

Melhores parâmetros encontrados:
{'num_bees': 300, 'limit': 50, 'max_iter': 70}
